In [ ]:
from typing import Any, ClassVar, Mapping, List
import pandas as pd
import numpy as np
import torch
import re
from pykeen.pipeline import pipeline
from pykeen.constants import DEFAULT_EMBEDDING_HPO_EMBEDDING_DIM_RANGE
from pykeen.typing import HeadRepresentation, RelationRepresentation, TailRepresentation
from pykeen.triples.triples_factory import TriplesFactory
from pykeen.triples.triples_factory import EntityMapping, RelationMapping
from pykeen.evaluation import RankBasedEvaluator
from pykeen.nn import Embedding, Interaction, Representation
from pykeen.models import ERModel
from pykeen.utils import negative_norm
from pykeen.metrics.ranking import InverseHarmonicMeanRank
from pykeen.training.callbacks import TrackerTrainingCallback
from torch.utils.tensorboard import SummaryWriter


C:\Users\SamuelMcmurray\.conda\envs\tf\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(
C:\Users\SamuelMcmurray\.conda\envs\tf\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\SamuelMcmurray\.conda\envs\tf\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [ ]:
INVERSE_SUFFIX = '_inverse'

train_path = "data\\train.tsv"
valid_path = "data\\valid.tsv"
test_path = "data\\test.tsv"

_num_entities = 7601
_num_relations = 28

writer = SummaryWriter()

In [ ]:
def create_entity_mapping(triples) -> EntityMapping:
    """Create mapping from entity labels to IDs.

    :param triples: shape: (n, 3), dtype: str
    """
    # Split triples
    heads, tails = triples[:, 0], triples[:, 2]
    # Sorting ensures consistent results when the triples are permuted
    entity_labels = sorted(set(heads).union(tails))
    # Create mapping
    return {
        str(label): i
        for (i, label) in enumerate(entity_labels)
    }

In [ ]:
def create_relation_mapping(relations: set) -> RelationMapping:
    """Create mapping from relation labels to IDs.

    :param relations: set
    """
    # Sorting ensures consistent results when the triples are permuted
    relation_labels = sorted(
        set(relations),
        key=lambda x: (re.sub(f'{INVERSE_SUFFIX}$', '', x), x.endswith(f'{INVERSE_SUFFIX}')),
    )
    # Create mapping
    return {
        str(label): i
        for (i, label) in enumerate(relation_labels)
    }

In [ ]:
def get_triples_and_relations():
    names = ['head', 'relation', 'tail']
    train = pd.read_csv(train_path, sep='\t', header=None, names=names)
    valid = pd.read_csv(valid_path, sep='\t', header=None, names=names)
    test = pd.read_csv(test_path, sep='\t', header=None, names=names)

    df = pd.concat([train, valid, test],axis=0).reset_index(drop=True)
    return df.values, set(df['relation'])

In [ ]:
triples, relations = get_triples_and_relations()

entity_to_id = create_entity_mapping(triples)
relation_to_id = create_relation_mapping(relations)

In [ ]:
names = ['head', 'relation', 'tail']

train = TriplesFactory.from_path(path=train_path,entity_to_id=entity_to_id, relation_to_id=relation_to_id)
train._num_entities = _num_entities
train._num_relations = _num_relations

valid = TriplesFactory.from_path(path=valid_path, entity_to_id=entity_to_id, relation_to_id=relation_to_id)
valid._num_entities = _num_entities
valid._num_relations = _num_relations

test = TriplesFactory.from_path(path=test_path, entity_to_id=entity_to_id, relation_to_id=relation_to_id)
test._num_entities = _num_entities
test._num_relations = _num_relations
evaluator = RankBasedEvaluator(metrics=InverseHarmonicMeanRank)

In [ ]:
class MyTrainingCallback(TrackerTrainingCallback):
    def post_epoch(self, epoch: int, epoch_loss: float, **kwargs: Any) -> None:
        vocab = train.entity_labeling.all_labels()
        entity_representation_modules: List['Representation'] = self.model.entity_representations
        relation_representation_modules: List['Representation'] = self.model.relation_representations

        entity_embeddings: Embedding = entity_representation_modules[0]
        relation_embeddings: Embedding = relation_representation_modules[0]

        entity_embedding_tensor: torch.FloatTensor = entity_embeddings()
        relation_embedding_tensor: torch.FloatTensor = relation_embeddings()

        writer.add_embedding(entity_embedding_tensor, vocab, global_step=epoch)
        writer.close()
        # import io
        # out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
        # out_m = io.open('metadata.tsv', 'w', encoding='utf-8')
        # for index, word in enumerate(vocab):
        #     vec = entity_embedding_tensor[index]
        #     out_v.write('\t'.join([str(x) for x in vec]) + "\n")
        #     out_m.write(word + "\n")
        # out_v.close()
        # out_m.close()

In [ ]:
model = "TransE"
dir_name = f'pharmaKG_{model}_Base'

result_base = pipeline(
    model=model,
    model_kwargs=dict(embedding_dim= 300),
    training=train,
    validation=valid,
    testing=test,
    random_seed=42,
    training_kwargs=dict(
        num_epochs= 300,
        batch_size=32,
        callbacks=MyTrainingCallback,
        checkpoint_frequency=0,
        checkpoint_name="transe_base.pt",
        checkpoint_directory="transe_base_checkpoints/chkpt_dir",
        checkpoint_on_failure=True,),# 30
    stopper='early',
    stopper_kwargs={'frequency': 10, 'patience': 1},
    result_tracker='tensorboard',
    result_tracker_kwargs=dict(experiment_path='tb-logs/transe_base'),
    evaluation_kwargs={"batch_size": 32},
    optimizer_kwargs={'lr': 0.1},
    evaluator=evaluator,
)
result_base.save_to_directory(dir_name)

No cuda devices were available. The model runs on CPU
INFO:pykeen.training.training_loop:=> loading checkpoint 'transe_base_checkpoints\chkpt_dir\transe_base.pt'
INFO:pykeen.training.training_loop:=> loaded checkpoint 'transe_base_checkpoints\chkpt_dir\transe_base.pt' stopped after having finished epoch 30
INFO:pykeen.stoppers.stopper:=> loading stopper summary dict from training loop checkpoint in 'transe_base_checkpoints\chkpt_dir\transe_base.pt'
INFO:pykeen.stoppers.stopper:=> loaded stopper summary dictionary from checkpoint in 'transe_base_checkpoints\chkpt_dir\transe_base.pt'


Evaluating on cpu:   0%|          | 0.00/49.8k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 285.42s seconds
INFO:pykeen.triples.triples_factory:Stored TriplesFactory(num_entities=7262, num_relations=28, create_inverse_triples=False, num_triples=386768, path="C:\Users\SamuelMcmurray\Documents\SchoolStuff\JU\DeepLearning\Assignment2\data\train.tsv") to file:///C:/Users/SamuelMcmurray/Documents/SchoolStuff/JU/DeepLearning/Assignment2/pharmaKG_TransE_Base/training_triples
INFO:pykeen.pipeline.api:Saved to directory: file:///C:/Users/SamuelMcmurray/Documents/SchoolStuff/JU/DeepLearning/Assignment2/pharmaKG_TransE_Base


In [ ]:
print("MRR: ", result_base.metric_results.get_metric("inverse_harmonic_mean_rank"))
print(result_base.metric_results.data)

MRR:  0.0032195772510021925
{('inverse_harmonic_mean_rank', 'head', 'optimistic'): 0.004630310882862159, ('inverse_harmonic_mean_rank', 'tail', 'optimistic'): 0.004831444107404305, ('inverse_harmonic_mean_rank', 'both', 'optimistic'): 0.004730877495133233, ('inverse_harmonic_mean_rank', 'head', 'realistic'): 0.003115967847406864, ('inverse_harmonic_mean_rank', 'tail', 'realistic'): 0.0033231868874281645, ('inverse_harmonic_mean_rank', 'both', 'realistic'): 0.0032195772510021925, ('inverse_harmonic_mean_rank', 'head', 'pessimistic'): 0.002728697428337269, ('inverse_harmonic_mean_rank', 'tail', 'pessimistic'): 0.0029327034270404465, ('inverse_harmonic_mean_rank', 'both', 'pessimistic'): 0.002830700427688857, ('median_rank', 'head', 'optimistic'): 3050.0, ('median_rank', 'tail', 'optimistic'): 2902.0, ('median_rank', 'both', 'optimistic'): 2976.0, ('median_rank', 'head', 'realistic'): 3063.0, ('median_rank', 'tail', 'realistic'): 2929.0, ('median_rank', 'both', 'realistic'): 2997.0, ('med

In [13]:
# import io
# best_embedding_dim = 50
# best_batch_size = 32
# best_lr = 0.1
# best_mrr = -0.1
# lr_array = [ 0.00001, 0.0001, 0.001, 0.01, 0.1]
#
# for embedding_dim in range(200, 800, 100):
#     for batch_size in range(32,128,32):
#         for lr in lr_array:
#             print(embedding_dim, batch_size, lr)
#             result_hyper_tune = pipeline(
#                 model="TransE",
#                 model_kwargs={"embedding_dim": embedding_dim},
#                 training=train,
#                 validation=valid,
#                 testing=test,
#                 random_seed=42,
#                 optimizer_kwargs={'lr': lr},
#                 training_kwargs=dict(
#                     num_epochs=1,
#                     batch_size=batch_size,
#                     ),  # 30
#                 evaluation_kwargs={'batch_size': 256},
#                 evaluator=evaluator,
#             )
#             mrr = result_hyper_tune.metric_results.get_metric("inverse_harmonic_mean_rank")
#             if mrr > best_mrr:
#                 best_embedding_dim = embedding_dim
#                 best_batch_size = batch_size
#                 best_lr = lr
#                 best_mrr = mrr
#                 out_h = io.open('Best_hyper_params.tsv', 'w', encoding='utf-8')
#
#                 out_h.write('\t' + str(best_embedding_dim) +'\t' + str(best_batch_size) + '\t' + str(best_lr) +"\n")
#
#                 out_h.close()
#
#
# print(f"Best embedding dim = {best_embedding_dim}, Best batch size = {best_batch_size}, Best learning rate = {best_lr}")


In [14]:
result_hyper = pipeline(
    model='TransE',
    model_kwargs={"embedding_dim": 200},
    training=train,
    validation=valid,
    testing=test,
    random_seed=42,
    optimizer_kwargs={'lr': 0.001},
    training_kwargs=dict(
        callbacks=MyTrainingCallback,
        num_epochs=300,
        batch_size= 96,
        checkpoint_frequency=0,
        checkpoint_name="transe_hyper.pt",
        checkpoint_directory="transe_hyper_checkpoints/chkpt_dir",
        checkpoint_on_failure=True,),  # 30
    stopper='early',
    stopper_kwargs={'frequency': 10, 'patience': 1},
    evaluation_kwargs={'batch_size': 32},
    evaluator=evaluator,
)

INFO:pykeen.pipeline.api:loaded random seed 42 from checkpoint.
INFO:pykeen.pipeline.api:Using device: None
INFO:pykeen.stoppers.early_stopping:Inferred checkpoint path for best model weights: C:\Users\SamuelMcmurray\.data\pykeen\checkpoints\best-model-weights-fd95d5b1-dc71-4c1a-a877-239dd6237c65.pt
INFO:pykeen.training.training_loop:=> loading checkpoint 'transe_hyper_checkpoints\chkpt_dir\transe_hyper.pt'
INFO:pykeen.training.training_loop:=> loaded checkpoint 'transe_hyper_checkpoints\chkpt_dir\transe_hyper.pt' stopped after having finished epoch 30
INFO:pykeen.stoppers.stopper:=> loading stopper summary dict from training loop checkpoint in 'transe_hyper_checkpoints\chkpt_dir\transe_hyper.pt'
INFO:pykeen.stoppers.stopper:=> loaded stopper summary dictionary from checkpoint in 'transe_hyper_checkpoints\chkpt_dir\transe_hyper.pt'


Evaluating on cpu:   0%|          | 0.00/49.8k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 214.11s seconds


In [15]:
print(result_hyper.metric_results.get_metric("inverse_harmonic_mean_rank"))
print(result_hyper.metric_results.data)
result_hyper.save_to_directory('TransE_hyper')

0.055973369628190994
{('inverse_harmonic_mean_rank', 'head', 'optimistic'): 0.055014938090823515, ('inverse_harmonic_mean_rank', 'tail', 'optimistic'): 0.0569318077254276, ('inverse_harmonic_mean_rank', 'both', 'optimistic'): 0.05597337290812557, ('inverse_harmonic_mean_rank', 'head', 'realistic'): 0.05501493811607361, ('inverse_harmonic_mean_rank', 'tail', 'realistic'): 0.05693180114030839, ('inverse_harmonic_mean_rank', 'both', 'realistic'): 0.055973369628190994, ('inverse_harmonic_mean_rank', 'head', 'pessimistic'): 0.05501492043350243, ('inverse_harmonic_mean_rank', 'tail', 'pessimistic'): 0.05693179490562237, ('inverse_harmonic_mean_rank', 'both', 'pessimistic'): 0.05597335766956239, ('median_rank', 'head', 'optimistic'): 158.0, ('median_rank', 'tail', 'optimistic'): 129.0, ('median_rank', 'both', 'optimistic'): 143.0, ('median_rank', 'head', 'realistic'): 158.0, ('median_rank', 'tail', 'realistic'): 129.0, ('median_rank', 'both', 'realistic'): 143.0, ('median_rank', 'head', 'pess

INFO:pykeen.triples.triples_factory:Stored TriplesFactory(num_entities=7262, num_relations=28, create_inverse_triples=False, num_triples=386768, path="C:\Users\SamuelMcmurray\Documents\SchoolStuff\JU\DeepLearning\Assignment2\data\train.tsv") to file:///C:/Users/SamuelMcmurray/Documents/SchoolStuff/JU/DeepLearning/Assignment2/TransE_hyper/training_triples
INFO:pykeen.pipeline.api:Saved to directory: file:///C:/Users/SamuelMcmurray/Documents/SchoolStuff/JU/DeepLearning/Assignment2/TransE_hyper


In [16]:
def stranse_interaction(
        h: torch.FloatTensor,
        r: torch.FloatTensor,
        r_h: torch.FloatTensor,
        r_t: torch.FloatTensor,
        t: torch.FloatTensor,
) -> torch.FloatTensor:
    r"""Evaluate the STransE interaction function.

    .. math ::
        f(h, r, t) = -\|R_h h - R_t t\|

    :param h: shape: (`*batch_dims`, dim)
        The head representations.
    :param r: shape: (`*batch_dims`, dim)
        The relation representations.
    :param r_h: shape: (`*batch_dims`, rel_dim, dim)
        The relation-specific head projection.
    :param r_t: shape: (`*batch_dims`, rel_dim, dim)
        The relation-specific tail projection.
    :param t: shape: (`*batch_dims`, dim)
        The tail representations.
    :param p:
        The p for the norm. cf. :func:`torch.linalg.vector_norm`.
    :param power_norm:
        Whether to return the powered norm.

    :return: shape: batch_dims
        The scores.
    """

    # SE interaction function
    # ||Wr,1h − Wr,2t||l1/2; Wr,1, Wr,2 ∈ R k×k
    # negative_norm(einsum("...rd,...d->...r", r_h, h) - einsum("...rd,...d->...r", r_t, t),p=p,power_norm=power_norm,)

    # TransE interaction function
    # ||h + r − t||l1/2; r ∈ R^k
    # negative_norm_of_sum(h, r, -t, p=p, power_norm=power_norm)
    # STranse ineteraction function
    # ||Wr,1h + r − Wr,2t||l1/2 ; Wr,1, Wr,2 ∈ R k×k; r ∈ R
    # print(einsum("j, ik->ik", r_h, h))
    return negative_norm((torch.einsum("...j, ...ij->...ij", r_h, h) + r) - torch.einsum("...j, ...ij->...ij", r_t, t), p=1,power_norm=False)

def prepare_hrt_for_functional(
    h: HeadRepresentation,
    r: RelationRepresentation,
    t: TailRepresentation,
):
    return h, t, r, r[0], r[1]

In [17]:
class STransEInteraction(Interaction):

    def forward(
        self,
        h: HeadRepresentation,
        r: RelationRepresentation,
        t: TailRepresentation,
    ) -> torch.FloatTensor:
        h, t, r, r_h, r_t = prepare_hrt_for_functional(h,r,t)
        result = stranse_interaction(h,r, r_h, r_t, t)

        return result



class STransE(ERModel):
    hpo_default: ClassVar[Mapping[str, Any]] = dict(
        embedding_dim=DEFAULT_EMBEDDING_HPO_EMBEDDING_DIM_RANGE,
        scoring_fct_norm=dict(type=int, low=1, high=2),
    )

    def __init__(
            self,
            *,
            embedding_dim: int = 50,
            scoring_fct_norm: int = 1,
            **kwargs,
    ) -> None:
        super().__init__(
            interaction=STransEInteraction(),
            interaction_kwargs=dict(p=scoring_fct_norm),
            entity_representations=Embedding,
            entity_representations_kwargs=dict(
                embedding_dim=embedding_dim,
            ),
            # Define the relation representations the same as the entities
            relation_representations=Embedding,
            relation_representations_kwargs=dict(
                embedding_dim=embedding_dim,
            ),
            **kwargs,
        )

In [18]:
result_stranse = pipeline(
    model=STransE,
    model_kwargs=dict(embedding_dim=300),
    training=train,
    validation=valid,
    testing=test,
    random_seed=42,
    training_kwargs=dict(
        num_epochs= 300,
        callbacks=MyTrainingCallback,
        batch_size=32,
        checkpoint_frequency=0,
        checkpoint_name="stranse_base.pt",
        checkpoint_directory="stranse_base_checkpoints/chkpt_dir",
        checkpoint_on_failure=True,),  # 30
    stopper='early',
    stopper_kwargs=dict(frequency= 10, patience= 1),
    optimizer_kwargs={'lr': 0.1},
    evaluator=evaluator,
)

INFO:pykeen.pipeline.api:loaded random seed 42 from checkpoint.
INFO:pykeen.pipeline.api:Using device: None
INFO:pykeen.stoppers.early_stopping:Inferred checkpoint path for best model weights: C:\Users\SamuelMcmurray\.data\pykeen\checkpoints\best-model-weights-85f45dbd-624b-4886-b840-ba14f82bbd69.pt
INFO:pykeen.training.training_loop:=> loading checkpoint 'stranse_base_checkpoints\chkpt_dir\stranse_base.pt'
INFO:pykeen.training.training_loop:=> loaded checkpoint 'stranse_base_checkpoints\chkpt_dir\stranse_base.pt' stopped after having finished epoch 140
INFO:pykeen.stoppers.stopper:=> loading stopper summary dict from training loop checkpoint in 'stranse_base_checkpoints\chkpt_dir\stranse_base.pt'
INFO:pykeen.stoppers.stopper:=> loaded stopper summary dictionary from checkpoint in 'stranse_base_checkpoints\chkpt_dir\stranse_base.pt'
INFO:pykeen.evaluation.evaluator:Currently automatic memory optimization only supports GPUs, but you're using a CPU. Therefore, the batch_size will be set 

Evaluating on cpu:   0%|          | 0.00/49.8k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 372.49s seconds


In [19]:
print(result_stranse.metric_results.get_metric("inverse_harmonic_mean_rank"))
print(result_stranse.metric_results.data)
result_stranse.save_to_directory(f'pharmaKG_STransE_base')

0.027064232155680656
{('inverse_harmonic_mean_rank', 'head', 'optimistic'): 0.02649018209103006, ('inverse_harmonic_mean_rank', 'tail', 'optimistic'): 0.02763829357930074, ('inverse_harmonic_mean_rank', 'both', 'optimistic'): 0.0270642378351654, ('inverse_harmonic_mean_rank', 'head', 'realistic'): 0.026490174233913418, ('inverse_harmonic_mean_rank', 'tail', 'realistic'): 0.027638288214802742, ('inverse_harmonic_mean_rank', 'both', 'realistic'): 0.027064232155680656, ('inverse_harmonic_mean_rank', 'head', 'pessimistic'): 0.026490164144528962, ('inverse_harmonic_mean_rank', 'tail', 'pessimistic'): 0.027638282558967425, ('inverse_harmonic_mean_rank', 'both', 'pessimistic'): 0.02706422335174819, ('median_rank', 'head', 'optimistic'): 521.0, ('median_rank', 'tail', 'optimistic'): 463.0, ('median_rank', 'both', 'optimistic'): 490.5, ('median_rank', 'head', 'realistic'): 521.0, ('median_rank', 'tail', 'realistic'): 463.0, ('median_rank', 'both', 'realistic'): 490.5, ('median_rank', 'head', 'p

INFO:pykeen.triples.triples_factory:Stored TriplesFactory(num_entities=7262, num_relations=28, create_inverse_triples=False, num_triples=386768, path="C:\Users\SamuelMcmurray\Documents\SchoolStuff\JU\DeepLearning\Assignment2\data\train.tsv") to file:///C:/Users/SamuelMcmurray/Documents/SchoolStuff/JU/DeepLearning/Assignment2/pharmaKG_STransE_base/training_triples
INFO:pykeen.pipeline.api:Saved to directory: file:///C:/Users/SamuelMcmurray/Documents/SchoolStuff/JU/DeepLearning/Assignment2/pharmaKG_STransE_base


In [20]:
result_stranse_hyper = pipeline(
    model=STransE,
    model_kwargs=dict(embedding_dim=200),
    training=train,
    validation=valid,
    testing=test,
    random_seed=42,
    training_kwargs=dict(
        num_epochs= 300,
        callbacks=MyTrainingCallback,
        batch_size=96,
        checkpoint_frequency=0,
        checkpoint_name="stranse_hyper.pt",
        checkpoint_directory="stranse_hyper_checkpoints/chkpt_dir",
        checkpoint_on_failure=True,),  # 30
    stopper='early',
    stopper_kwargs=dict(frequency= 10, patience= 1),
    optimizer_kwargs={'lr': 0.001},
    evaluator=evaluator,
)

INFO:pykeen.pipeline.api:loaded random seed 42 from checkpoint.
INFO:pykeen.pipeline.api:Using device: None
INFO:pykeen.stoppers.early_stopping:Inferred checkpoint path for best model weights: C:\Users\SamuelMcmurray\.data\pykeen\checkpoints\best-model-weights-dd3b39c4-e6d0-4de1-bed2-ef438a82027d.pt
INFO:pykeen.training.training_loop:=> loading checkpoint 'stranse_hyper_checkpoints\chkpt_dir\stranse_hyper.pt'
INFO:pykeen.training.training_loop:=> loaded checkpoint 'stranse_hyper_checkpoints\chkpt_dir\stranse_hyper.pt' stopped after having finished epoch 100
INFO:pykeen.stoppers.stopper:=> loading stopper summary dict from training loop checkpoint in 'stranse_hyper_checkpoints\chkpt_dir\stranse_hyper.pt'
INFO:pykeen.stoppers.stopper:=> loaded stopper summary dictionary from checkpoint in 'stranse_hyper_checkpoints\chkpt_dir\stranse_hyper.pt'
INFO:pykeen.evaluation.evaluator:Currently automatic memory optimization only supports GPUs, but you're using a CPU. Therefore, the batch_size will

Evaluating on cpu:   0%|          | 0.00/49.8k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 276.68s seconds


In [21]:
print(result_stranse_hyper.metric_results.get_metric("inverse_harmonic_mean_rank"))
print(result_stranse_hyper.metric_results.data)
result_stranse_hyper.save_to_directory(f'pharmaKG_STransE_hyper')

0.04369758442044258
{('inverse_harmonic_mean_rank', 'head', 'optimistic'): 0.04132072333810905, ('inverse_harmonic_mean_rank', 'tail', 'optimistic'): 0.046074458525835096, ('inverse_harmonic_mean_rank', 'both', 'optimistic'): 0.04369759093197207, ('inverse_harmonic_mean_rank', 'head', 'realistic'): 0.04132070764899254, ('inverse_harmonic_mean_rank', 'tail', 'realistic'): 0.04607445374131203, ('inverse_harmonic_mean_rank', 'both', 'realistic'): 0.04369758442044258, ('inverse_harmonic_mean_rank', 'head', 'pessimistic'): 0.04132069200917406, ('inverse_harmonic_mean_rank', 'tail', 'pessimistic'): 0.04607445047675227, ('inverse_harmonic_mean_rank', 'both', 'pessimistic'): 0.04369757124296316, ('median_rank', 'head', 'optimistic'): 242.0, ('median_rank', 'tail', 'optimistic'): 190.0, ('median_rank', 'both', 'optimistic'): 214.0, ('median_rank', 'head', 'realistic'): 242.0, ('median_rank', 'tail', 'realistic'): 190.0, ('median_rank', 'both', 'realistic'): 214.0, ('median_rank', 'head', 'pessi

INFO:pykeen.triples.triples_factory:Stored TriplesFactory(num_entities=7262, num_relations=28, create_inverse_triples=False, num_triples=386768, path="C:\Users\SamuelMcmurray\Documents\SchoolStuff\JU\DeepLearning\Assignment2\data\train.tsv") to file:///C:/Users/SamuelMcmurray/Documents/SchoolStuff/JU/DeepLearning/Assignment2/pharmaKG_STransE_hyper/training_triples
INFO:pykeen.pipeline.api:Saved to directory: file:///C:/Users/SamuelMcmurray/Documents/SchoolStuff/JU/DeepLearning/Assignment2/pharmaKG_STransE_hyper


In [25]:
# log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, update_freq=1)
models = [result_base, result_hyper, result_stranse, result_stranse_hyper]
for index, result_model in enumerate(models):
    model = result_model.model

    entity_representation_modules: List['Representation'] = model.entity_representations
    relation_representation_modules: List['Representation'] = model.relation_representations

    entity_embeddings: Embedding = entity_representation_modules[0]
    relation_embeddings: Embedding = relation_representation_modules[0]

    entity_embedding_tensor: torch.FloatTensor = entity_embeddings()
    relation_embedding_tensor: torch.FloatTensor = relation_embeddings()

    vocab = train.entity_labeling.all_labels()

    writer.add_embedding(entity_embedding_tensor, vocab, global_step=index)
    writer.close()
# import io
# out_v = io.open('runs/tensors.tsv', 'w', encoding='utf-8')
# out_m = io.open('runs/metadata.tsv', 'w', encoding='utf-8')
# for index, word in enumerate(vocab):
#     vec = entity_embedding_tensor[index]
#     out_v.write('\t'.join([str(x) for x in vec]) + "\n")
#     out_m.write(word + "\n")
# out_v.close()
# out_m.close()

In [2]:
%reload_ext tensorboard
%tensorboard --logdir 'runs/Apr30_14-45-04_DESKTOP-1DMSUFS'

Reusing TensorBoard on port 6006 (pid 992), started 0:09:00 ago. (Use '!kill 992' to kill it.)